<a href="https://colab.research.google.com/github/Junaeid-Shoaib/AI_Agents/blob/main/Multi_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1) Install with extras while PINNING smolagents to 1.20.0
!pip install -q "smolagents[litellm]==1.20.0" "plotly>=6.1.1" "kaleido>=1.0.0" geopandas shapely


# 3) Verify versions after restart
import smolagents, plotly, shapely, geopandas
print("smolagents:", smolagents.__version__)
print("plotly:", plotly.__version__)
print("geopandas:", geopandas.__version__)
print("shapely:", shapely.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 988.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 37.7 MB/s eta 0:00:00
smolagents: 1.20.0
plotly: 6.3.0
geopandas: 1.1.1
shapely: 2.1.1


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [ ]:
!pip install --quiet smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents langfuse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 996.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

In [ ]:
!pip install --quiet -U huggingface_hub openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('pk')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('sk')
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # EU region

from langfuse import get_client
langfuse = get_client()

if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
SmolagentsInstrumentor().instrument()


Langfuse client is authenticated and ready!


In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('secretKey')


In [ ]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool

from smolagents import CodeAgent, OpenAIServerModel



In [ ]:
task = """Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in colchestser, 51.9147° N latitude and 0.9473° E longitude), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations"""

In [ ]:

from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = userdata.get('serpK')

In [ ]:
!pip install markdownify requests
# agent = CodeAgent(
#     model=model,
#     tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
#     additional_authorized_imports=["pandas"],
#     max_steps=20,
# )
#model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")
model = OpenAIServerModel(model_id="gpt-4o", temperature=0.0)
agent = CodeAgent(tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time], model=model, additional_authorized_imports=["pandas"], max_steps=8, verbosity_level=2)


In [ ]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in colchestser, 51.9147° N latitude and 0.9473° E longitude), and return them to me as a pandas     │
│ dataframe.                                                                                                      │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:393: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:402: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To solve this task, I will first search for a list of Harry Potter filming locations worldwide. Then, I   
will calculate the travel time from Colchester to each of these locations using a passenger plane. After that, I   
will search for nearby cricket stadiums for each filming location. Finally, I will compile all this information    
into a pandas DataFrame and return it.                                                                             
                                                                                                                   
Let's start by searching for Harry Potter filming locations worldwide.                                             
                                                                                                                   
<code>                                                                                                             
locations_search = web_search(query="Harry Potter filming locations worldwide")                                    
print(locations_search)                                                                                            
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  locations_search = web_search(query="Harry Potter filming locations worldwide")                                  
  print(locations_search)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I have gathered information about nearby cricket stadiums for each Harry Potter filming location. Now, I  
will compile all the information, including the filming locations, travel times, and nearby cricket stadiums, into 
a pandas DataFrame and return it.                                                                                  
                                                                                                                   
<code>                                                                                                             
import pandas as pd                                                                                                
                                                                                                                   
# Create a DataFrame with the collected data                                                                       
data = {                                                                                                           
    "Location": list(corrected_location_coords.keys()),                                                            
    "Travel Time (hours)": list(travel_times.values()),                                                            
    "Nearby Cricket Stadiums": [                                                                                   
        "Lord's Cricket Ground, London",                                                                           
        "Lord's Cricket Ground, London",                                                                           
        "Lord's Cricket Ground, London",                                                                           
        "Gloucestershire County Cricket Club, Gloucester",                                                         
        "Lacock Cricket Club, Lacock",                                                                             
        "Sussex County Cricket Club, East Sussex",                                                                 
        "Christ Church Ground, Oxford",                                                                            
        "Riverside Ground, Durham",                                                                                
        "Alnwick Cricket Club, Northumberland",                                                                    
        "West of Scotland Cricket Club, Glasgow",                                                                  
        "No nearby cricket stadium found"                                                                          
    ]                                                                                                              
}                                                                                                                  
                                                                                                                   
df = pd.DataFrame(data)                                                                                            
final_answer(df)                                                                                                   
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Create a DataFrame with the collected data                                                                     
  data = {                                                                                                         
      "Location": list(corrected_location_coords.keys()),                                                          
      "Travel Time (hours)": list(travel_times.values()),                                                          
      "Nearby Cricket Stadiums": [                                                                                 
          "Lord's Cricket Ground, London",                                                                         
          "Lord's Cricket Ground, London",                                                                         
          "Lord's Cricket Ground, London",                                                                         
          "Gloucestershire County Cricket Club, Gloucester",                                                       
          "Lacock Cricket Club, Lacock",                                                                           
          "Sussex County Cricket Club, East Sussex",                                                               
          "Christ Church Ground, Oxford",                                                                          
          "Riverside Ground, Durham",                                                                              
          "Alnwick Cricket Club, Northumberland",                                                                  
          "West of Scotland Cricket Club, Glasgow",                                                                
          "No nearby cricket stadium found"                                                                        
      ]                                                                                                            
  }                                                                                                                
                                                                                                                   
  df = pd.DataFrame(data)                                                                                          
  final_answer(df)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer:                                    Location  Travel Time (hours)  \
0                        London Zoo, London                 1.11   
1                 Leadenhall Market, London                 1.10   
2              King's Cross Station, London                 1.10   
3          Gloucester Cathedral, Gloucester                 1.27   
4                      Lacock Abbey, Lacock                 1.27   
5   Seven Sisters chalk cliffs, East Sussex                 1.17   
6             Christ Church College, Oxford                 1.19   
7                  Durham Cathedral, Durham                 1.44   
8            Alnwick Castle, Northumberland                 1.52   
9    Glenfinnan Viaduct, Scottish Highlands                 1.84   
10                 Cliffs of Moher, Ireland                 1.87   

                            Nearby Cricket Stadiums  
0                     Lord's Cricket Ground, London  
1                     Lord's Cricket Ground, London  
2                     Lord's Cricket Ground, London  
3   Gloucestershire County Cricket Club, Gloucester  
4                       Lacock Cricket Club, Lacock  
5           Sussex County Cricket Club, East Sussex  
6                      Christ Church Ground, Oxford  
7                          Riverside Ground, Durham  
8              Alnwick Cricket Club, Northumberland  
9            West of Scotland Cricket Club, Glasgow  
10                  No nearby cricket stadium found  

[Step 6: Duration 5.29 seconds| Input tokens: 45,027 | Output tokens: 1,326]

In [ ]:
result

,Location,Travel Time (hours),Nearby Cricket Stadiums
0,London,1.10,## Search Results\n0. [Lord's Cricket Ground |...
1,Gloucester Cathedral,1.26,## Search Results\n0. [Gloucestershire Cricket...
2,Lacock Abbey,1.26,## Search Results\n0. [Lord's Cricket Ground (...
3,Seven Sisters,1.17,## Search Results\n0. [Sussex County Cricket C...
4,"Christ Church College, Oxford",1.18,## Search Results\n0. [Christ Church Ground](h...
5,Durham Cathedral,1.44,## Search Results\n0. [Riverside Ground](https...
6,Alnwick Castle,1.52,## Search Results\n0. [Durham Cricket - Sports...
7,Scottish Highlands,1.82,## Search Results\n0. [West of Scotland Cricke...
8,Cliffs of Moher,1.87,## Search Results\n0. [MALAHIDE CRICKET CLUB (...


In [ ]:
agent.planning_interval = 3

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in colchestser, 51.9147° N latitude and 0.9473° E longitude), and return them to me as a pandas     │
│ dataframe.                                                                                                      │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- The task is to find all Harry Potter filming locations in the world.
- The task requires calculating the travel time from Colchester (51.9147° N, 0.9473° E) to each filming location 
using a passenger plane.
- The task also requires finding cricket stadiums near the filming locations.

### 1.2. Facts to look up
- A list of all Harry Potter filming locations worldwide. This can be found through a web search or specific 
film-related databases or websites.
- Coordinates (latitude and longitude) for each Harry Potter filming location. This information can be found on the
same sources as the filming locations.
- Information on nearby cricket stadiums for each filming location. This can be found through a web search or 
sports-related databases or websites.
- Coordinates (latitude and longitude) for each cricket stadium. This information can be found on the same sources 
as the cricket stadiums.

### 1.3. Facts to derive
- Calculate the travel time from Colchester to each Harry Potter filming location using the coordinates and a 
typical passenger plane cruising speed.
- Determine the proximity of cricket stadiums to the filming locations to identify which ones are nearby.

## 2. Plan

1. Perform a web search to find a comprehensive list of all Harry Potter filming locations worldwide.
2. For each filming location found, visit the source webpage to confirm the information and obtain the coordinates 
(latitude and longitude).
3. Calculate the travel time from Colchester to each filming location using the coordinates and a typical passenger
plane cruising speed.
4. For each filming location, perform a web search to find nearby cricket stadiums.
5. Visit the source webpages for the cricket stadiums to confirm their information and obtain their coordinates.
6. Determine which cricket stadiums are nearby each filming location based on proximity.
7. Compile the data into a pandas dataframe, including the filming location name, coordinates, travel time from 
Colchester, and nearby cricket stadiums.
8. Provide the final answer by returning the pandas dataframe.
9. Use the `final_answer` function to output the dataframe.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Let's start by performing a web search to find a comprehensive list of all Harry Potter filming locations          
worldwide. This will be our first step in gathering the necessary data.                                            
                                                                                                                   
<code>                                                                                                             
filming_locations_search = web_search(query="Harry Potter filming locations worldwide")                            
print(filming_locations_search)                                                                                    
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  filming_locations_search = web_search(query="Harry Potter filming locations worldwide")                          
  print(filming_locations_search)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

[Step 5: Duration 81.37 seconds| Input tokens: 85,915 | Output tokens: 1,266]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-469MiyZXoCsFHBxwysnQ046z 
on tokens per min (TPM): Limit 30000, Requested 40108. The input or output tokens must be reduced in order to run 
successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': 
None, 'code': 'rate_limit_exceeded'}}

[Step 6: Duration 3.27 seconds]

AgentGenerationError: Error in generating model output:
Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-469MiyZXoCsFHBxwysnQ046z on tokens per min (TPM): Limit 30000, Requested 40108. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

Splitting the task between two agents

In [ ]:
os.environ["SERPER_API_KEY"] = userdata.get('serperKey') # Changed to SERPER_API_KEY and getting from userdata

# model = InferenceClientModel(
#     "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
# )
model = OpenAIServerModel(model_id="gpt-4o", temperature=0.0, max_tokens=8096)
web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

Manager

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel
import os
from PIL import Image

def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(

model=OpenAIServerModel("gpt-4o", max_tokens=8096),
   tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
    "geopandas",
    "plotly",
    "plotly.io",  # explicitly add this
    "plotly.express",  # explicitly add this too
    "shapely",
    "json",
    "pandas",
    "numpy",
    "kaleido"
],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [ ]:
manager_agent.visualize()

CodeAgent | gpt-4o
├── ✅ Authorized imports: ['geopandas', 'plotly', 'plotly.io', 'plotly.express', 'shapely', 'json', 'pandas', 
│   'numpy', 'kaleido']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4o
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.

In [ ]:
!apt-get update -qq
!apt-get install -y chromium-browser


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor libfuse3-3 libudev1 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser libfuse3-3 snapd squashfs-tools systemd-hwe-hwdb
  udev
The following packages will be upgraded:
  libudev1
1 upgraded, 7 newly installed, 0 to remove and 36 not upgraded.
Need to get 30.3 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main a

In [ ]:
!which chromium-browser


/usr/bin/chromium-browser


In [ ]:
import os
os.environ["PATH"] += os.pathsep + "/usr/bin/chromium-browser"


In [ ]:
!pip install markdownify requests
!pip install -U kaleido
import plotly.io as pio



manager_agent.run("""
Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in colchestser, 51.9147° N latitude and 0.9473° E longitude), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in colchestser, 51.9147° N latitude and 0.9473° E longitude), and return them to me as a pandas     │
│ dataframe.                                                                                                      │
│ Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points │
│ in total.                                                                                                       │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- The current location is Colchester, with coordinates: 51.9147° N latitude and 0.9473° E longitude.

### 1.2. Facts to look up
1. List of all Harry Potter filming locations worldwide.
   - Source: Use the web to find this information.
2. Coordinates of each Harry Potter filming location.
   - Source: Use the web to find this information.
3. Cricket stadiums located near the Harry Potter filming locations.
   - Source: Use the web to find this information.
4. Coordinates of each cricket stadium found near the filming locations.
   - Source: Use the web to find this information.

### 1.3. Facts to derive
1. Calculate the travel time from Colchester to each of the Harry Potter filming locations using the passenger 
plane's cruising speed.
2. Calculate the travel time from Colchester to each of the nearby cricket stadiums using the passenger plane's 
cruising speed.

## 2. Plan

1. Use the web_agent to search for and retrieve a list of all Harry Potter filming locations worldwide, including 
identifying their coordinates.
2. Use the web_agent to search for cricket stadiums near each of the identified filming locations, including 
identifying their coordinates.
3. Use the `calculate_cargo_travel_time` function to compute the travel time from Colchester to each of the filming
locations.
4. Use the `calculate_cargo_travel_time` function to compute the travel time from Colchester to each of the cricket
stadiums found.
5. Compile the filming locations and nearby cricket stadiums along with their travel times into a pandas dataframe.
6. Create a spatial map using Plotly, where each filming location and cricket stadium is represented as a scatter 
point. The points should be colored based on their travel times.
7. Save the generated map as "saved_map.png".
8. Display the map and provide it as the final answer using the `final_answer` function.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Let's start by searching for all the Harry Potter filming locations worldwide and their coordinates. For that, I'll
use the web_agent to find a comprehensive list.                                                                    

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
Here is your code snippet:
Let's start by searching for all the Harry Potter filming locations worldwide and their coordinates. For that, I'll
use the web_agent to find a comprehensive list.</code>
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
<code>
# Your python code here
</code>
Make sure to provide correct code blobs.

...

[Step 15: Duration 46.58 seconds| Input tokens: 177,783 | Output tokens: 4,179]

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:393: FutureWarning:

Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:402: FutureWarning:

Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.



Reached max steps.

[Step 16: Duration 52.56 seconds| Input tokens: 198,023 | Output tokens: 4,814]

'Due to the current platform\'s limitations on saving images, completing the task with an actual saved image file isn\'t possible here. However, the DataFrame with all the required information is prepared and can be used to generate and save the plot on your local environment. Below is the step-by-step process for achieving this task in a suitable environment.\n\nPlan:\n\n1. **Prepare the DataFrame:**\n   - The DataFrame with Harry Potter filming locations and their calculated travel times from Colchester is provided.\n\n2. **Generate and Save the Map Locally:**\n   - To create and save the spatial map as an .png image on your local machine, please follow these steps:\n\n### Code to Execute Locally:\n\n```python\nimport plotly.express as px\nimport pandas as pd\n\n# DataFrame\nlocations_df = pd.DataFrame({\n    \'Location\': [\n        "Alnwick Castle, England",\n        "Gloucester Cathedral, England",\n        "Durham Cathedral, England",\n        "Leadenhall Market, London, England"

In [ ]:
manager_agent.python_executor.state["fig"]